# Project 4

In this project, you will summarize and present your analysis from Projects 1-3.

### Intro: Write a problem Statement/ Specific Aim for this project

Answer: Determine the impacts that GPA, GRE scores, and alma mater presige all have (or don't have) on acceptance of students into graduate school.

### Dataset:  Write up a description of your data and any cleaning that was completed

In [39]:
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pylab as pl
import numpy as np
%matplotlib inline

In [40]:
#Read in data from source 
df = pd.read_csv("../my_hw_repo/admissions.csv")

#display the header
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


In [41]:
df = df.dropna() #Drop entries with null values

In [42]:
dfnew = pd.get_dummies(df, columns=['prestige'], drop_first=False ) #Get dummies for prestige, and keep all of them for optics

In [43]:
df.describe()

,admit,gre,gpa,prestige
count,397.000000,397.000000,397.000000,397.000000
mean,0.317380,587.858942,3.392242,2.488665
std,0.466044,115.717787,0.380208,0.947083
min,0.000000,220.000000,2.260000,1.000000
25%,0.000000,520.000000,3.130000,2.000000
50%,0.000000,580.000000,3.400000,2.000000
75%,1.000000,660.000000,3.670000,3.000000
max,1.000000,800.000000,4.000000,4.000000


In [44]:
dfnew.describe()

,admit,gre,gpa,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
count,397.000000,397.000000,397.000000,397.000000,397.000000,397.000000,397.000000
mean,0.317380,587.858942,3.392242,0.153652,0.372796,0.304786,0.168766
std,0.466044,115.717787,0.380208,0.361070,0.484159,0.460898,0.375017
min,0.000000,220.000000,2.260000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,520.000000,3.130000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,580.000000,3.400000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,660.000000,3.670000,0.000000,1.000000,1.000000,0.000000
max,1.000000,800.000000,4.000000,1.000000,1.000000,1.000000,1.000000


Answer: I deleted all entries with any null values. This left me with 397 students in the data. Then I created dummy variables for the prestige variable. I could then see that the average student got about a 588 on the gre, with a 3.39 GPA. 15% of the students in the sample went to schools of prestige 1, 37% went to prestige 2, 30% went to prestige 3, and 17% went to prestige 4. 

### Demo: Provide a table that explains the data by admission status

Mean (STD) or counts by admission status for each variable 

| Not Admitted | Admitted
---| ---|---
GPA | mean(std)  | mean(std)
GRE |mean(std) | mean(std)
Prestige 1 | frequency (%) | frequency (%)
Prestige 2 | frequency (%) | frequency (%)
Prestige 3 |frequency (%) | frequency (%)
Prestige 4 |frequency (%) | frequency (%)

In [53]:
print dfnew.loc[dfnew.admit == 0, 'gpa'].mean()
print dfnew.loc[dfnew.admit == 1, 'gpa'].mean()
print dfnew.loc[dfnew.admit == 0, 'gre'].mean()
print dfnew.loc[dfnew.admit == 1, 'gre'].mean()

3.34715867159
3.48920634921
573.579335793
618.571428571


In [55]:
#Get crosstab of admitted vs. not, keep Totals so can divide by it to get percentages - is there a better way?
admit_prestige = pd.crosstab(df['admit'], df['prestige'], rownames=['admit'], margins = True)
admit_prestige = admit_prestige/admit_prestige.ix["All"]

#Cut the Totals out of the dataframe
admit_prestige.iloc[:2,:4]

prestige,1.0,2.0,3.0,4.0
admit,,,,
0,0.459016,0.641892,0.768595,0.820896
1,0.540984,0.358108,0.231405,0.179104


NOTE: I DON'T KNOW HOW TO GET THIS ALL IN ONE TABLE. IS THERE A QUICK WAY, OR A METHOD THAT I AM FORGETTING OR MISSED? PLEASE ADVISE

### Methods: Write up the methods used in your analysis

In [58]:
df_final = pd.get_dummies(df, columns=['prestige'], drop_first=True) #Drop first now
df_final['intercept'] = 1.0 #manually add intercept
train_cols = df_final.columns[1:]

#Fit the model using logistic regression
logit = sm.Logit(df_final['admit'], df_final[train_cols])
admit_fit = logit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


In [59]:
admit_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Mon, 20 Mar 2017   Pseudo R-squ.:                 0.08166
Time:                        20:51:43   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
================================================================================
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa              0.7793      0.333      2.344      0.019         0.128     1.431
prestige_2.0    -0.6801      0.317     -2.146      0.032        -1.301    -0.059
prestige_3.0    -1.3387      0.345     -3.882      0.000        -2.015    -0.663
prestige_4.0    -1.5534      0.417     -3.721      0.000        -2.372    -0.735
intercept       -3.8769      1.142     -3.393      0.001        -6.116    -1.638
================================================================================
"""

Answer: I used a logistic regression to fit this model.

### Results: Write up your results

In [60]:
print np.exp(admit_fit.params)

gre             1.002221
gpa             2.180027
prestige_2.0    0.506548
prestige_3.0    0.262192
prestige_4.0    0.211525
intercept       0.020716
dtype: float64


Answer: After fitting the model, I list the odds ratios of the variables. I can see that each additional point of GRE improves the chance of getting in by 0.2%, and that each point of GPA improves the chance of getting in by 118%. Students applying from prestige 1 schools have a large advantage (as seen before, they have a 54% chance), and each drop in prestige makes it considerably tougher to get admitted.

### Discussion: Write up your discussion and future steps

Answer: In the future, I may try different methods and see which give the best results. I could run a random forest and get the order of importance of the variables. 